# Semantic Search - OHS Legislation (Turkish)


### Import the Needed Packages

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys

In [4]:
sys.path.append('../../../../DLAIUtils')

In [5]:
#!pip install python-dotenv
#!pip install datasets==2.15.0
#!pip install PyPDF2
#!pip install openai
#!pip install google-generativeai
!pip install -U google-generativeai

In [6]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from DLAIUtils import Utils
import os
import time
import torch
from dotenv import load_dotenv
import PyPDF2
from openai import OpenAI
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

True

In [7]:
from tqdm.auto import tqdm

### Load the OHS Legislation PDF Dataset

In [8]:
# Load ISG Mevzuat PDF
pdf_path = 'data/isg_mevzuat.pdf'

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)
print(f"Total characters extracted: {len(pdf_text)}")
print(f"First 500 characters:\n{pdf_text[:500]}")

Total characters extracted: 234007
First 500 characters:
Bu doküman Global Belgelendirme Hizmetleri A.Ş. tarafından 01.06.2021 tarihinde oluşturulmuştur.  
  
 
 
 
 
 
 
 
 
4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  ...........................................................................................................  4 
Alt İşverenlik Yönetmeliği  .......................................................................................


In [9]:
# Improved text chunking to preserve complete sentences
def split_text_into_chunks(text, chunk_size=800, overlap=200):
    """Split text into overlapping chunks at sentence boundaries"""
    import re
    
    # Split into sentences (Turkish-aware)
    sentence_endings = r'[.!?]\s+'
    sentences = re.split(sentence_endings, text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        # If adding this sentence exceeds chunk_size and we have content, save chunk
        if len(current_chunk) + len(sentence) > chunk_size and current_chunk:
            chunks.append(current_chunk.strip())
            # Keep overlap: start new chunk with last part of previous chunk
            words = current_chunk.split()
            overlap_words = words[-int(len(words) * 0.2):] if len(words) > 5 else []
            current_chunk = ' '.join(overlap_words) + ' ' + sentence
        else:
            current_chunk += ' ' + sentence
    
    # Add the last chunk
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    
    return chunks

# Create chunks from PDF text with better sentence preservation
text_chunks = split_text_into_chunks(pdf_text, chunk_size=800, overlap=200)
print(f"Number of text chunks: {len(text_chunks)}")
print(f"\nFirst 3 chunks:")
for i, chunk in enumerate(text_chunks[:3]):
    print(f"\n--- Chunk {i+1} (length: {len(chunk)}) ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)

Number of text chunks: 405

First 3 chunks:

--- Chunk 1 (length: 699) ---
Bu doküman Global Belgelendirme Hizmetleri A.Ş tarafından 01.06.2021 tarihinde oluşturulmuştur 4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  .....................................................

--- Chunk 2 (length: 779) ---
11 Çocuk ve Genç İşçilerin Çalıştırılma Usul ve Esasları Hakkında Yönetmelik ............................ 12 
Gebe veya Emziren Kadınların Çalıştırılma Şartlarıyla İlgili Yönetmelik  .................................. 13 
Haftalık İş Günlerine Bölünemeyen Çalışma Süreleri Yönetmeliği  .................

--- Chunk 3 (length: 751) ---
Çalışma Yönetmeliği .............................. 17 Kadın Ça lışanların Gece Postalarında Çalıştırılma Koşulları Hakkında Yönetmelik .................. 18 
Postalar Halinde İşçi Çalıştırılarak Yürütülen İşlerde Çalışmalara İlişkin Yönetmelik  ..........

In [10]:
# Clean and prepare ISG mevzuat chunks
def clean_text(text):
    # Replace problematic characters with ASCII equivalents
    text = text.replace('–', '-')  # en dash
    text = text.replace('—', '-')  # em dash
    text = text.replace(''', "'")  # left single quotation mark
    text = text.replace(''', "'")  # right single quotation mark
    text = text.replace('"', '"')  # left double quotation mark
    text = text.replace('"', '"')  # right double quotation mark
    # Keep Turkish characters but remove other problematic Unicode
    # Don't encode to ASCII for Turkish text
    return text

# Clean the chunks
cleaned_chunks = [clean_text(chunk) for chunk in text_chunks if len(chunk.strip()) > 50]

print(f'Number of cleaned chunks: {len(cleaned_chunks)}')
print('\nFirst 2 chunks:')
for i, chunk in enumerate(cleaned_chunks[:2]):
    print(f"\n--- Chunk {i+1} ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)

Number of cleaned chunks: 405

First 2 chunks:

--- Chunk 1 ---
Bu doküman Global Belgelendirme Hizmetleri A.Ş tarafından 01.06.2021 tarihinde oluşturulmuştur 4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  .....................................................

--- Chunk 2 ---
11 Çocuk ve Genç İşçilerin Çalıştırılma Usul ve Esasları Hakkında Yönetmelik ............................ 12 
Gebe veya Emziren Kadınların Çalıştırılma Şartlarıyla İlgili Yönetmelik  .................................. 13 
Haftalık İş Günlerine Bölünemeyen Çalışma Süreleri Yönetmeliği  .................


### Check cuda and Setup the model

**Note**: "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this course, we are using CPUs. So, you might notice some code cells taking a little longer to run.

We are using *all-MiniLM-L6-v2* sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [12]:
query = 'işçi sağlığı ve güvenliği nedir?'
xq = model.encode(query)
xq.shape

(384,)

In [13]:
utils = Utils()

### Setup Pinecone

In [14]:
# Load API keys from .env file
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Verify Pinecone API key is loaded
if not PINECONE_API_KEY or PINECONE_API_KEY == 'your_pinecone_api_key_here':
    raise ValueError("Please set PINECONE_API_KEY in .env file")

# Configure Google Gemini
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✓ API keys loaded from .env file (OpenAI + Google Gemini)")
else:
    print("✓ API keys loaded from .env file (OpenAI only)")

✓ API keys loaded from .env file (OpenAI + Google Gemini)


In [15]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = utils.create_dlai_index_name('isg')

# Check and delete existing index if present
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    print(f"Deleting existing index: {INDEX_NAME}")
    pinecone.delete_index(INDEX_NAME)
    time.sleep(5)  # Wait for deletion to complete

print(f"Creating new index: {INDEX_NAME}")
pinecone.create_index(
    name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

# Wait for index to be ready
print("Waiting for index to be ready...")
time.sleep(10)

# Connect to index
index = pinecone.Index(INDEX_NAME)
print(f"✓ Index ready: {index}")
print(f"✓ Index stats: {index.describe_index_stats()}")

Deleting existing index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Creating new index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Creating new index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Waiting for index to be ready...
Waiting for index to be ready...
✓ Index ready: <pinecone.db_data.index.Index object at 0x3070be350>
✓ Index ready: <pinecone.db_data.index.Index object at 0x3070be350>
✓ Index stats: {'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '150',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 14 Jan 2026 18:47:39 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '43',
                                    'x-pinecone-request-id': '5770634670681135803',
                                    'x-pinecone-req

### Create Embeddings and Upsert to Pinecone

In [16]:
batch_size = 100
# Use all chunks from ISG mevzuat
isg_chunks = cleaned_chunks

# Create a mapping to store the actual text (for later retrieval)
text_mapping = {i: text for i, text in enumerate(isg_chunks)}

print(f"Total chunks to process: {len(isg_chunks)}")

for i in tqdm(range(0, len(isg_chunks), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(isg_chunks))
    # create IDs batch
    ids = [f"isg_{x}" for x in range(i, i_end)]
    # create embeddings
    xc = model.encode(isg_chunks[i:i_end])
    # create metadata batch - store only chunk_id to avoid encoding issues
    metadatas = [{'chunk_id': idx} for idx in range(i, i_end)]
    # create records list for upsert with proper format
    records = [
        {
            'id': id_,
            'values': values.tolist(),
            'metadata': metadata
        }
        for id_, values, metadata in zip(ids, xc, metadatas)
    ]
    # upsert to Pinecone
    index.upsert(vectors=records)

print(f"\n✓ Successfully uploaded {len(isg_chunks)} ISG mevzuat chunks to Pinecone")
print(f"Text mapping created with {len(text_mapping)} entries")

Total chunks to process: 405


  0%|          | 0/5 [00:00<?, ?it/s]


✓ Successfully uploaded 405 ISG mevzuat chunks to Pinecone
Text mapping created with 405 entries


In [17]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '185',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 14 Jan 2026 18:47:49 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '4',
                                    'x-pinecone-request-id': '5054425798342982642',
                                    'x-pinecone-request-latency-ms': '3',
                                    'x-pinecone-response-duration-ms': '5'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 405}},
 'storageFullness': 0.0,
 'total_vector_count': 405,
 'vector_type': 'dense'}

### Run Your Queries - Multi-AI Provider Support

In [63]:
# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Enhanced query function with multiple AI providers and response length control
def run_query(query, use_ai=True, ai_provider="openai", use_reasoning=False, max_length="normal", top_k=5):
    """
    Query ISG legislation with semantic search and optional AI enhancement
    
    Args:
        query: The question to ask (in Turkish)
        use_ai: Whether to use AI for response synthesis (default: True)
        ai_provider: "openai", "gemini", or "none" (default: "openai")
        use_reasoning: Use advanced reasoning models (default: False)
        max_length: "short" (3-4 sentences), "medium" (5-7 sentences), or "normal" (detailed)
        top_k: Number of relevant chunks to retrieve (default: 5)
    """
    embedding = model.encode(query).tolist()
    results = index.query(top_k=top_k, vector=embedding, include_metadata=True, include_values=False)
    
    if not use_ai or ai_provider == "none":
        # Simple mode: just print the chunks
        print("🔍 Most Relevant Results:\n")
        for i, result in enumerate(results['matches'], 1):
            chunk_id = result['metadata']['chunk_id']
            text = text_mapping.get(chunk_id, "Text not found")
            print(f"{i}. [Score: {round(result['score'], 2)}]\n{text}\n")
    else:
        # AI mode: use AI to synthesize a better answer
        if ai_provider == "gemini":
            if use_reasoning:
                print("🧠 Deep Analysis with Gemini 2.5 Pro:\n")
            else:
                print("🤖 Enhanced Response with Gemini 2.5 Flash:\n")
        else:  # openai
            if use_reasoning:
                print("🧠 Deep Analysis with GPT-4 (o1 Reasoning):\n")
            else:
                print("🤖 Enhanced Response with GPT-4 mini:\n")
        
        # Gather relevant chunks
        relevant_texts = []
        for result in results['matches']:
            chunk_id = result['metadata']['chunk_id']
            text = text_mapping.get(chunk_id, "")
            if text:
                relevant_texts.append(text)
        
        context = "\n\n".join(relevant_texts)
        
        # Length-aware instructions
        if max_length == "short":
            length_instruction = "VERY IMPORTANT: Limit your response to maximum 3-4 sentences. Only mention the most important points."
            max_tokens = 2048
        elif max_length == "medium":
            length_instruction = "Keep your response concise (5-7 sentences)."
            max_tokens = 3072
        else:  # normal or long
            length_instruction = "Provide a detailed and complete response."
            max_tokens = 4096
        
        system_prompt = f"""You are an expert on OHS (Occupational Health and Safety) legislation. 
Answer questions based ONLY on the provided legislation texts. 
Provide answers in proper Turkish, in a bullet-point format, and reference relevant articles.
{length_instruction}"""

        user_prompt = f"""Answer the question based on the following OHS legislation texts.

Question: {query}

Legislation Texts:
{context}

Please:
1. Answer the question fully and clearly
2. Use ONLY the information from the provided legislation texts
3. Provide the answer in proper Turkish, in bullet-point format
4. Reference relevant articles
5. Give practical examples
6. Highlight important points
{length_instruction}"""

        # Call AI provider
        if ai_provider == "gemini":
            # Use Gemini 1.5 Flash for both normal and reasoning (FREE!)
            # Note: Using same model for both to avoid quota limits
            gemini_model = genai.GenerativeModel('models/gemini-2.5-flash')
            response = gemini_model.generate_content(
                f"{system_prompt}\n\n{user_prompt}",
                generation_config={
                    'temperature': 0.3,
                    'max_output_tokens': max_tokens,
                    'top_p': 0.95,
                    'top_k': 40
                }
            )
            if use_reasoning:
                print("💭 Used Gemini 2.5 Flash (with reasoning mode)\n")
            else:
                print("🤖 Enhanced Response with Gemini 2.5 Flash:\n")
            print(response.text)
        else:  # openai
            if use_reasoning:
                # Use o1 model for deep reasoning
                response = openai_client.chat.completions.create(
                    model="o1-mini",
                    messages=[
                        {"role": "user", "content": f"{system_prompt}\n\n{user_prompt}"}
                    ]
                )
                print("💭 Used O1 Reasoning Model (with internal reasoning)\n")
                print(response.choices[0].message.content)
            else:
                # Use GPT-4 for standard response
                response = openai_client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ],
                    temperature=0.3,
                    max_tokens=max_tokens
                )
                print(response.choices[0].message.content)
        
        print("\n" + "="*80)
        print(f"📚 Source Texts (showing top {min(3, len(relevant_texts))}):")
        for i, text in enumerate(relevant_texts[:3], 1):
            print(f"\n{i}. {text[:200]}...")
    
    return results

## 🧪 Test Soruları

Aşağıda İSG mevzuatı ile ilgili örnek sorular bulunmaktadır. Her hücre bağımsız olarak çalıştırılabilir.

In [ ]:
# Soru 1: Kimyasal maddelerle çalışma
query_1 = "Kimyasal maddelerle çalışılan bir işyerinde, çalışanların maruziyetini önlemek için alınması gereken öncelikli tedbirler (ikame, teknik önlemler vb.) sırasıyla nelerdir?"

run_query(query_1, use_ai=True, ai_provider="openai", max_length="long")

In [ ]:
# Soru 2: Risk değerlendirmesi süreci
query_2 = "Bir iş yerinde risk değerlendirmesi yapılırken hangi adımlar izlenmeli ve kimler bu ekibe dahil edilmelidir?"

run_query(query_2, use_ai=True, ai_provider="openai", max_length="long")

In [ ]:
# Soru 3: İşveren yükümlülükleri
query_3 = "İşverenin iş sağlığı ve güvenliği konusundaki genel yükümlülükleri maddeler halinde nelerdir?"

run_query(query_3, use_ai=True, ai_provider="openai", max_length="long")

In [ ]:
# Soru 4: İş kazası ve acil durum planı
query_4 = "İş yerlerinde acil durum planı hazırlanırken hangi hususlar dikkate alınmalı ve tatbikatlar ne sıklıkla yapılmalıdır?"

run_query(query_4, use_ai=True, ai_provider="openai", max_length="long")